In [4]:
pip install xlsxwriter

  Obtaining dependency information for xlsxwriter from https://files.pythonhosted.org/packages/3a/0c/3662f4a66880196a590b202f0db82d919dd2f89e99a27fadef91c4a33d41/xlsxwriter-3.2.9-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/175.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/175.3 kB ? eta -:--:--
   -- ------------------------------------- 10.2/175.3 kB ? eta -:--:--
   -- ------------------------------------- 10.2/175.3 kB ? eta -:--:--
   --------- ----------------------------- 41.0/175.3 kB 279.3 kB/s eta 0:00:01
   -------------------- ------------------ 92.2/175.3 kB 581.0 kB/s eta 0:00:01
   -------------------------------------- 175.3/175.3 kB 878.1 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [9]:
import nltk
from nltk import DownloadError

ImportError: cannot import name 'DownloadError' from 'nltk' (C:\Users\Natasha\anaconda3\Lib\site-packages\nltk\__init__.py)

In [25]:
# ---  CLASSIFICAÇÃO MANUAL ESTRATÉGICA ---
import pandas as pd
import numpy as np
import re
from collections import Counter

print("="*70)
print("🎯 400 CLASSIFICAÇÕES MANUAIS")
print("="*70)

# --- PASSO 1: ANALISAR O DATASET PARA ESCOLHER OS 400 MELHORES ---
print("📊 Analisando dataset para selecionar amostra estratégica...")

# Carregar dataset
df = pd.read_excel(r'C:\Users\Natasha\Documents\GITHUB\olist\olist_avaliacoes.xlsx')

# Filtrar apenas comentários válidos
df = df[df['Msg_comentário'].notna()]
df['Msg_comentário'] = df['Msg_comentário'].astype(str).str.strip()
df = df[df['Msg_comentário'] != '']

print(f"Comentários válidos: {len(df):,}")

# --- PASSO 2: ESTRATÉGIA DE SELEÇÃO INTELIGENTE ---
print("\n🔍 Selecionando amostra estratégica de 400 comentários...")

amostra_estratégica = []

# ESTRATÉGIA: Pegar exemplos de cada tipo de padrão

# 1. NEGATIVOS ÓBVIOS (100 exemplos) - Prioridade MÁXIMA
print("\n1. Coletando NEGATIVOS ÓBVIOS...")
padroes_negativos = [
    'não gost', 'péssim', 'ruim', 'horrível', 'decepcion', 
    'defeito', 'quebrado', 'não funciona', 'não recomendo',
    'atrasado', 'demorou', 'lento', 'caro demais', 'perda de dinheiro'
]

negativos_coletados = 0
for padrao in padroes_negativos:
    mask = df['Msg_comentário'].str.contains(padrao, case=False, na=False)
    exemplos = df[mask].sample(min(15, sum(mask)), random_state=42)
    
    for _, row in exemplos.iterrows():
        if negativos_coletados < 100:
            amostra_estratégica.append({
                'Comentario': row['Msg_comentário'],
                'Tipo_Coleta': 'NEGATIVO_OBVIO',
                'Classificacao_Sugerida': 'NEGATIVO',
                'Classificacao_Manual': ''
            })
            negativos_coletados += 1

print(f"   Negativos coletados: {negativos_coletados}")

# 2. POSITIVOS ÓBVIOS (100 exemplos)
print("\n2. Coletando POSITIVOS ÓBVIOS...")
padroes_positivos = [
    'excelente', 'perfeito', 'adorei', 'recomendo', 'ótimo',
    'maravilhoso', 'superou', 'gostei muito', 'valeu a pena',
    'rápida entrega', 'antes do prazo', 'qualidade excelente'
]

positivos_coletados = 0
for padrao in padroes_positivos:
    mask = df['Msg_comentário'].str.contains(padrao, case=False, na=False)
    exemplos = df[mask].sample(min(15, sum(mask)), random_state=42)
    
    for _, row in exemplos.iterrows():
        # Verificar se não tem negação antes
        texto = row['Msg_comentário'].lower()
        pos = texto.find(padrao)
        if pos > 0:
            texto_antes = texto[:pos]
            if any(neg in texto_antes for neg in ['não', 'nem', 'mas', 'porém']):
                continue  # Pular se tem negação
        
        if positivos_coletados < 100:
            amostra_estratégica.append({
                'Comentario': row['Msg_comentário'],
                'Tipo_Coleta': 'POSITIVO_OBVIO',
                'Classificacao_Sugerida': 'POSITIVO',
                'Classificacao_Manual': ''
            })
            positivos_coletados += 1

print(f"   Positivos coletados: {positivos_coletados}")

# 3. NEUTROS CLAROS (50 exemplos) - textos curtos factuais
print("\n3. Coletando NEUTROS CLAROS...")
neutros_coletados = 0
for _, row in df.iterrows():
    texto = str(row['Msg_comentário'])
    palavras = texto.split()
    
    # Textos curtos com palavras factuais
    if len(palavras) <= 6:
        palavras_factuais = ['recebi', 'chegou', 'entregue', 'produto', 'ok', 'certo']
        if any(pal in texto.lower() for pal in palavras_factuais):
            if neutros_coletados < 50:
                amostra_estratégica.append({
                    'Comentario': texto,
                    'Tipo_Coleta': 'NEUTRO_OBVIO',
                    'Classificacao_Sugerida': 'NEUTRO',
                    'Classificacao_Manual': ''
                })
                neutros_coletados += 1
    
    if neutros_coletados >= 50:
        break

print(f"   Neutros coletados: {neutros_coletados}")

# 4. CASOS DIFÍCEIS/AMBÍGUOS (150 exemplos) - onde o modelo erra
print("\n4. Coletando CASOS DIFÍCEIS...")

# Primeiro, usar resultados anteriores para pegar onde o modelo errou
try:
    resultados_anteriores = pd.read_excel(
        r'C:\Users\Natasha\Documents\GITHUB\olist\resultados_corrigidos_finais.xlsx',
        sheet_name='Apenas_Comentarios_Validos'
    )
    
    # Pegar casos onde confiança é baixa ou classificação é suspeita
    casos_dificeis = resultados_anteriores[
        (resultados_anteriores['Confianca'] < 0.6) |
        ((resultados_anteriores['Sentimento'] == 'POSITIVO') & 
         resultados_anteriores['Msg_comentário'].str.contains('não|problema|defeito', case=False, na=False)) |
        ((resultados_anteriores['Sentimento'] == 'NEGATIVO') & 
         resultados_anteriores['Msg_comentário'].str.contains('excelente|perfeito|adorei', case=False, na=False))
    ].sample(min(150, len(resultados_anteriores)), random_state=42)
    
    for _, row in casos_dificeis.iterrows():
        amostra_estratégica.append({
            'Comentario': row['Msg_comentário'],
            'Tipo_Coleta': 'CASO_DIFICIL',
            'Classificacao_Sugerida': '',  # Não sugerir - você decide
            'Classificacao_Manual': ''
        })
    
    print(f"   Casos difíceis coletados: {len(casos_dificeis)}")
    
except:
    # Se não tiver resultados anteriores, pegar aleatório
    casos_aleatorios = df.sample(150, random_state=42)
    for _, row in casos_aleatorios.iterrows():
        amostra_estratégica.append({
            'Comentario': row['Msg_comentário'],
            'Tipo_Coleta': 'ALEATORIO',
            'Classificacao_Sugerida': '',
            'Classificacao_Manual': ''
        })
    print(f"   Casos aleatórios coletados: 150")

# Criar DataFrame
df_amostra = pd.DataFrame(amostra_estratégica)
df_amostra = df_amostra.drop_duplicates(subset=['Comentario'])

print(f"\n📊 AMOSTRA FINAL SELECIONADA: {len(df_amostra)} comentários")

# --- PASSO 3: CRIAR ARQUIVO PARA CLASSIFICAÇÃO MANUAL ---
print("\n💾 Criando arquivo para classificação manual...")

caminho_classificacao = r'C:\Users\Natasha\Documents\GITHUB\olist\classificacao_manual_400.xlsx'

with pd.ExcelWriter(caminho_classificacao, engine='xlsxwriter') as writer:
    # Aba principal
    df_amostra.to_excel(writer, sheet_name='Classificar', index=False)
    
    # Aba de instruções detalhadas
    instrucoes = pd.DataFrame({
        'INSTRUÇÕES DETALHADAS': [
            '='*70,
            '🎯 CLASSIFIQUE 400 COMENTÁRIOS MANUALMENTE',
            '='*70,
            '',
            '📋 REGRAS CLARAS PARA CLASSIFICAÇÃO:',
            '',
            '🔴 NEGATIVO (cliente INSATISFEITO):',
            '   • "não gostei", "péssimo", "ruim", "horrível"',
            '   • "não recomendo", "não indico"',
            '   • "defeito", "quebrado", "não funciona"',
            '   • "atrasou", "demorou muito", "caro demais"',
            '   • Reclamações, problemas, insatisfação',
            '',
            '🟢 POSITIVO (cliente SATISFEITO):',
            '   • "excelente", "perfeito", "ótimo", "adorei"',
            '   • "recomendo", "superou expectativas"',
            '   • Elogios, satisfação, vai comprar de novo',
            '   • ⚠️ CUIDADO: "excelente MAS..." pode ser negativo!',
            '',
            '⚪ NEUTRO (apenas INFORMATIVO):',
            '   • "recebi o produto", "chegou hoje"',
            '   • "tudo certo", "conforme pedido"',
            '   • Fatos sem emoção, apenas informação',
            '   • Textos curtos (até 6 palavras) factuais',
            '',
            '🎯 EXEMPLOS PRÁTICOS:',
            '1. "Produto veio com defeito" → NEGATIVO',
            '2. "Adorei, recomendo!" → POSITIVO',
            '3. "Recebi hoje" → NEUTRO',
            '4. "Excelente mas demorou" → NEGATIVO (tem "mas")',
            '5. "Não gostei, qualidade ruim" → NEGATIVO',
            '6. "Chegou antes do prazo" → POSITIVO',
            '',
            '📝 COMO CLASSIFICAR:',
            '1. Leia o comentário',
            '2. Identifique a EMOÇÃO do cliente',
            '3. Use as regras acima',
            '4. Preencha "Classificacao_Manual" com:',
            '   - POSITIVO (🟢)',
            '   - NEGATIVO (🔴)',
            '   - NEUTRO (⚪)',
            '',
            '⏰ TEMPO ESTIMADO: 60-90 minutos',
            '💡 DICA: Classifique em 2 sessões de 30-45 min'
        ]
    })
    instrucoes.to_excel(writer, sheet_name='Instrucoes', index=False)
    
    # Aba com exemplos já classificados
    exemplos_classificados = pd.DataFrame({
        'Comentario': [
            'não gostei do produto',
            'excelente qualidade, recomendo',
            'recebi o produto hoje',
            'veio com defeito, não funciona',
            'chegou atrasado mas gostei',
            'péssimo, nunca mais compro',
            'tudo certo com a entrega',
            'produto bom pelo preço'
        ],
        'Classificacao_Correta': [
            'NEGATIVO',
            'POSITIVO',
            'NEUTRO',
            'NEGATIVO',
            'POSITIVO',  # Termina positivo apesar do problema
            'NEGATIVO',
            'NEUTRO',
            'POSITIVO'
        ],
        'Explicacao': [
            'Claramente insatisfeito',
            'Claramente satisfeito e recomenda',
            'Apenas informativo, sem emoção',
            'Reclamação de defeito',
            'Problema mas ficou satisfeito no final',
            'Muito insatisfeito',
            'Apenas informativo',
            'Satisfeito com custo-benefício'
        ]
    })
    exemplos_classificados.to_excel(writer, sheet_name='Exemplos', index=False)

print(f"✅ Arquivo criado: {caminho_classificacao}")

🎯 ESTRATÉGIA DEFINITIVA: 400 CLASSIFICAÇÕES MANUAIS
📊 Analisando dataset para selecionar amostra estratégica...
Comentários válidos: 40,950

🔍 Selecionando amostra estratégica de 400 comentários...

1. Coletando NEGATIVOS ÓBVIOS...
   Negativos coletados: 100

2. Coletando POSITIVOS ÓBVIOS...
   Positivos coletados: 100

3. Coletando NEUTROS CLAROS...
   Neutros coletados: 50

4. Coletando CASOS DIFÍCEIS...
   Casos difíceis coletados: 150

📊 AMOSTRA FINAL SELECIONADA: 391 comentários

💾 Criando arquivo para classificação manual...
✅ Arquivo criado: C:\Users\Natasha\Documents\GITHUB\olist\classificacao_manual_400.xlsx


In [27]:
# --- CÓDIGO DE EXECUÇÃO ---
import pandas as pd
import pickle
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from nltk.corpus import stopwords # Importar nltk

# Você pode precisar baixar as stopwords se for a primeira vez
# import nltk
# nltk.download('stopwords') 

print("\n" + "="*70)
print("🤖 TREINANDO NOVO MODELO COM AS CLASSIFICAÇÕES MANUAIS (400+)")
print("="*70)

# 1. Carregar suas 400 classificações
caminho_400 = r'C:\Users\Natasha\Documents\GITHUB\olist\classificacao_manual_400.xlsx'
try:
    df_classificado = pd.read_excel(
        caminho_400,
        sheet_name='Classificar'
    )
except FileNotFoundError:
    print(f"❌ Erro: Arquivo de classificação manual não encontrado em: {caminho_400}")
    # Você pode querer sair do script aqui
    exit()

df_classificado = df_classificado[df_classificado['Classificacao_Manual'].notna()]
df_classificado['Classificacao_Manual'] = df_classificado['Classificacao_Manual'].str.strip().str.upper()

print(f"✅ Classificações válidas encontradas: {len(df_classificado)}")

# 2. Adicionar validações anteriores (se quiser)
caminho_validacoes_antigas = r'C:\Users\Natasha\Documents\GITHUB\olist\validacao_manual_final.xlsx'
try:
    validacoes_anteriores = pd.read_excel(
        caminho_validacoes_antigas,
        sheet_name='Classificar'
    )
    validacoes_anteriores = validacoes_anteriores[validacoes_anteriores['Classificacao_Manual'].notna()]
    
    # Combinar
    df_treino = pd.concat([
        # Renomeia para 'Msg_comentário' para combinar
        df_classificado.rename(columns={'Comentario': 'Msg_comentário'}), 
        validacoes_anteriores[['Msg_comentário', 'Classificacao_Manual']]
    ], ignore_index=True).drop_duplicates(subset=['Msg_comentário']) # Remove duplicatas
    
    print(f"Total de amostras de treino (400 + antigas): {len(df_treino)}")
except:
    print("⚠️ Não foi possível carregar validações anteriores. Usando apenas as 400.")
    df_treino = df_classificado.rename(columns={'Comentario': 'Msg_comentário'})

# 3. Pré-processamento (função simples)
def preprocessar_simples(textos):
    processados = []
    for texto in textos:
        texto = str(texto).lower()
        # Nota: O uso de \\s no re.sub está correto dentro de f-strings ou strings longas.
        texto = re.sub(r'[^a-záàâãéêíóôõúüç0-9\s]', ' ', texto) 
        texto = re.sub(r'\s+', ' ', texto).strip()
        processados.append(texto)
    return processados

# 4. Treinar modelo
X = preprocessar_simples(df_treino['Msg_comentário'].values)
y = df_treino['Classificacao_Manual'].values

# TfidfVectorizer aprimorado
try:
    stop_words = stopwords.words('portuguese')
except:
    stop_words = []
    print("⚠️ Stopwords não encontradas. Continuando sem.")

vectorizer = TfidfVectorizer(
    max_features=1500, # Aumentado para melhor representação
    ngram_range=(1, 2),
    stop_words=stop_words,
    min_df=2,
    max_df=0.9
)
X_vec = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_vec, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y # Estratificação é crucial para bases pequenas
)

model = SVC(
    kernel='linear', 
    probability=True, 
    random_state=42, 
    class_weight='balanced',
    C=1.0 # Parâmetro de regularização
)
model.fit(X_train, y_train)

# 5. Avaliar
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"✅ Acurácia do novo modelo (Teste/Validação): {accuracy:.1%}")

# 6. Salvar modelo
modelo_final = {
    'model': model,
    'vectorizer': vectorizer,
    'acuracia': accuracy,
    'tamanho_treino': len(df_treino),
    'distribuicao_classes': df_treino['Classificacao_Manual'].value_counts().to_dict(),
    'data_treinamento': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
}

caminho_modelo_final = r'C:\Users\Natasha\Documents\GITHUB\olist\modelo_400_classificacoes.pkl'
with open(caminho_modelo_final, 'wb') as f:
    pickle.dump(modelo_final, f)

print(f"💾 Modelo salvo em: {caminho_modelo_final}")

# 7. Aplicar a todo dataset
caminho_dataset_completo = r'C:\Users\Natasha\Documents\GITHUB\olist\olist_avaliacoes.xlsx'
try:
    df_completo = pd.read_excel(caminho_dataset_completo)
    print(f"\nTotal de linhas no dataset completo: {len(df_completo)}")
except FileNotFoundError:
    print(f"❌ Erro: Dataset completo não encontrado em: {caminho_dataset_completo}")
    exit()

# Classificar em lote para eficiência
textos_completos = df_completo['Msg_comentário'].fillna('').astype(str).values
textos_proc_completos = preprocessar_simples(textos_completos)

X_todo = vectorizer.transform(textos_proc_completos)
predicoes = model.predict(X_todo)
probabilidades = model.predict_proba(X_todo)
confiancas = np.max(probabilidades, axis=1)

# Adicionar ao DataFrame
df_completo['Sentimento'] = predicoes
df_completo['Confianca'] = confiancas

# Ajustar os comentários vazios/ausentes
mask_vazios = (df_completo['Msg_comentário'].isna()) | (df_completo['Msg_comentário'].str.strip() == '')
df_completo.loc[mask_vazios, 'Sentimento'] = 'NEUTRO_VAZIO'
df_completo.loc[mask_vazios, 'Confianca'] = 1.0

# 8. Salvar resultados
caminho_final_resultados = r'C:\Users\Natasha\Documents\GITHUB\olist\resultados_finais_400_treinados.xlsx'
df_completo.to_excel(caminho_final_resultados, index=False)

print(f"✅ Resultados finais salvos: {caminho_final_resultados}")

# Estatísticas Finais
print("\n" + "="*70)
print("📊 ESTATÍSTICAS FINAIS")
print("="*70)

# Distribuição dos comentários válidos (excluindo neutro vazio)
distrib = df_completo[df_completo['Sentimento'] != 'NEUTRO_VAZIO']['Sentimento'].value_counts()
print(f"Distribuição dos {len(distrib):,} comentários classificados:")
total_classificado = distrib.sum()

for cat, qtd in distrib.items():
    perc = qtd / total_classificado * 100
    print(f"  {cat}: {qtd:,} ({perc:.1f}%)")

conf_media = df_completo[df_completo['Sentimento'] != 'NEUTRO_VAZIO']['Confianca'].mean()
print(f"\n🎭 Confiança Média das Classificações: {conf_media:.1%}")

print("\n" + "="*70)
print("🎉 PROCESSO CONCLUÍDO COM SUCESSO!")
print("======================================================================\n")


🤖 TREINANDO NOVO MODELO COM AS CLASSIFICAÇÕES MANUAIS (400+)
✅ Classificações válidas encontradas: 391
Total de amostras de treino (400 + antigas): 584
✅ Acurácia do novo modelo (Teste/Validação): 80.3%
💾 Modelo salvo em: C:\Users\Natasha\Documents\GITHUB\olist\modelo_400_classificacoes.pkl

Total de linhas no dataset completo: 99224
✅ Resultados finais salvos: C:\Users\Natasha\Documents\GITHUB\olist\resultados_finais_400_treinados.xlsx

📊 ESTATÍSTICAS FINAIS
Distribuição dos 3 comentários classificados:
  NEGATIVO: 18,724 (45.7%)
  POSITIVO: 18,295 (44.7%)
  NEUTRO: 3,931 (9.6%)

🎭 Confiança Média das Classificações: 76.2%

🎉 PROCESSO CONCLUÍDO COM SUCESSO!



In [28]:
# --- CÓDIGO DE EXECUÇÃO HÍBRIDO ---
import pandas as pd
import pickle
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from nltk.corpus import stopwords 

print("\n" + "="*70)
print("🤖 TREINANDO E APLICANDO MODELO HÍBRIDO (ML + REGRAS DEFINITIVAS)")
print("="*70)

# --- FUNÇÃO DE REGRAS DEFINITIVAS (SEU CÓDIGO) ---
def classificar_manual_corretamente(texto, classe_ml, confianca_ml):
    """
    REGRAS DEFINITIVAS para correção (OVERRIDE) do modelo ML
    """
    texto_lower = str(texto).lower()
    
    # 1. ERROS CRÍTICOS (MÁXIMA PRIORIDADE)
    # Se cliente diz explicitamente NÃO GOSTOU = SEMPRE NEGATIVO
    if any(phrase in texto_lower for phrase in ['não gost', 'não recomendo', 'insatisfeit']):
        return 'NEGATIVO', 'Regra_Critica', max(confianca_ml, 0.99)
    
    # Se tem problema/defeito = SEMPRE NEGATIVO
    if any(word in texto_lower for word in ['defeito', 'quebrado', 'amassado', 'rasgado', 'não funciona', 'problema']):
        return 'NEGATIVO', 'Regra_Critica', max(confianca_ml, 0.99)
    
    # 2. PROBLEMAS NA LOGÍSTICA (Alta Prioridade)
    # Se tem atraso/demorou = SEMPRE NEGATIVO
    if 'demorou' in texto_lower or 'atraso' in texto_lower or 'atrasou' in texto_lower or 'atrasada' in texto_lower:
        # Se for um problema, mas terminar positivamente ("mas chegou bem")
        if 'mas' in texto_lower:
            partes = texto_lower.split('mas')
            if len(partes) > 1 and any(pos in partes[-1] for pos in ['bom', 'valeu', 'gostei', 'ok', 'recomendo']):
                return 'POSITIVO', 'Regra_Mas', max(confianca_ml, 0.90)
        return 'NEGATIVO', 'Regra_Atraso', max(confianca_ml, 0.95)
    
    # Se recebeu errado/faltando = SEMPRE NEGATIVO
    if any(phrase in texto_lower for phrase in ['veio errado', 'faltando', 'não recebi', 'diferente do']):
        return 'NEGATIVO', 'Regra_Erro', max(confianca_ml, 0.95)
    
    # 3. ELOGIOS FORTES (Override POSITIVO)
    if any(word in texto_lower for word in ['recomendo', 'excelente', 'perfeito', 'adorei', 'ótimo', 'maravilhoso']):
        # Evitar classificação positiva se houver negação muito próxima
        if not any(neg in texto_lower for neg in ['não', 'nem', 'mas']):
             return 'POSITIVO', 'Regra_Elogio', max(confianca_ml, 0.99)

    # 4. DEFAULT: Textos curtos e neutros
    palavras = texto_lower.split()
    if len(palavras) <= 5:
        if any(pal in texto_lower for pal in ['recebi', 'chegou', 'entregue', 'aguardando', 'produto']):
            if not any(neg in texto_lower for neg in ['não', 'nada']):
                # Se o ML não for neutro e a confiança for baixa, forçar NEUTRO
                if classe_ml != 'NEUTRO' and confianca_ml < 0.8:
                    return 'NEUTRO', 'Regra_Curto', max(confianca_ml, 0.80)

    # REGRA 7: Se termina com "mas gostei" = POSITIVO (apesar do problema)
    if 'mas' in texto_lower and 'gostei' in texto_lower.split('mas')[-1]:
        return 'POSITIVO', 'Regra_Mas_Pos', max(confianca_ml, 0.90)

    # Se nenhuma regra se aplica, confiar na classificação do ML
    return classe_ml, 'ML', confianca_ml


# --- RESTANTE DO CÓDIGO (1 a 6: TREINAMENTO DO ML) ---
# ... (Manter os passos 1 a 6 sem alteração) ...

# 1. Carregar suas 400 classificações
caminho_400 = r'C:\Users\Natasha\Documents\GITHUB\olist\classificacao_manual_400.xlsx'
try:
    df_classificado = pd.read_excel(caminho_400, sheet_name='Classificar')
except FileNotFoundError:
    print(f"❌ Erro: Arquivo de classificação manual não encontrado em: {caminho_400}")
    exit()

# ... (Manter o restante da Fase 1 e 2: Carregamento e combinação de datasets) ...
df_classificado = df_classificado[df_classificado['Classificacao_Manual'].notna()]
df_classificado['Classificacao_Manual'] = df_classificado['Classificacao_Manual'].str.strip().str.upper()

caminho_validacoes_antigas = r'C:\Users\Natasha\Documents\GITHUB\olist\validacao_manual_final.xlsx'
try:
    validacoes_anteriores = pd.read_excel(caminho_validacoes_antigas, sheet_name='Classificar')
    validacoes_anteriores = validacoes_anteriores[validacoes_anteriores['Classificacao_Manual'].notna()]
    df_treino = pd.concat([
        df_classificado.rename(columns={'Comentario': 'Msg_comentário'}), 
        validacoes_anteriores[['Msg_comentário', 'Classificacao_Manual']]
    ], ignore_index=True).drop_duplicates(subset=['Msg_comentário'])
except:
    df_treino = df_classificado.rename(columns={'Comentario': 'Msg_comentário'})
print(f"Total de amostras de treino para ML: {len(df_treino)}")

# 3. Pré-processamento (função simples)
def preprocessar_simples(textos):
    processados = []
    for texto in textos:
        texto = str(texto).lower()
        texto = re.sub(r'[^a-záàâãéêíóôõúüç0-9\s]', ' ', texto) 
        texto = re.sub(r'\s+', ' ', texto).strip()
        processados.append(texto)
    return processados

# 4. Treinar modelo
X = preprocessar_simples(df_treino['Msg_comentário'].values)
y = df_treino['Classificacao_Manual'].values
try:
    stop_words = stopwords.words('portuguese')
except:
    stop_words = []
vectorizer = TfidfVectorizer(max_features=1500, ngram_range=(1, 2), stop_words=stop_words, min_df=2, max_df=0.9)
X_vec = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.2, random_state=42, stratify=y)
model = SVC(kernel='linear', probability=True, random_state=42, class_weight='balanced', C=1.0)
model.fit(X_train, y_train)

# 5. Avaliar e 6. Salvar modelo
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"✅ Acurácia do ML puro (Teste/Validação): {accuracy:.1%}")

modelo_final = {'model': model, 'vectorizer': vectorizer, 'acuracia': accuracy, 'tamanho_treino': len(df_treino), 'data_treinamento': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}
caminho_modelo_final = r'C:\Users\Natasha\Documents\GITHUB\olist\modelo_400_classificacoes_hibrido.pkl'
with open(caminho_modelo_final, 'wb') as f:
    pickle.dump(modelo_final, f)
print(f"💾 Modelo ML puro salvo em: {caminho_modelo_final}")


# --- PASSO 7: APLICAÇÃO E CORREÇÃO HÍBRIDA ---
print("\n" + "="*70)
print("⚙️ APLICANDO MODELO HÍBRIDO (ML + REGRAS) AO DATASET COMPLETO")
print("="*70)

caminho_dataset_completo = r'C:\Users\Natasha\Documents\GITHUB\olist\olist_avaliacoes.xlsx'
try:
    df_completo = pd.read_excel(caminho_dataset_completo)
except FileNotFoundError:
    print(f"❌ Erro: Dataset completo não encontrado em: {caminho_dataset_completo}")
    exit()

# 7.1. Classificação pelo ML Puro
textos_completos = df_completo['Msg_comentário'].fillna('').astype(str).values
textos_proc_completos = preprocessar_simples(textos_completos)

X_todo = vectorizer.transform(textos_proc_completos)
predicoes_ml = model.predict(X_todo)
probabilidades = model.predict_proba(X_todo)
confiancas_ml = np.max(probabilidades, axis=1)

df_completo['Sentimento_ML'] = predicoes_ml
df_completo['Confianca_ML'] = confiancas_ml

# 7.2. Aplicação do Override pelas Regras Híbridas
resultados_hibridos = []
for index, row in df_completo.iterrows():
    texto = row['Msg_comentário']
    classe_ml = row['Sentimento_ML']
    confianca_ml = row['Confianca_ML']
    
    # Aplica a Regra de Override
    classe_final, metodo_final, confianca_final = classificar_manual_corretamente(
        texto, classe_ml, confianca_ml
    )
    resultados_hibridos.append((classe_final, metodo_final, confianca_final))

# Adicionar resultados corrigidos ao DataFrame
df_completo['Sentimento_Final'] = [r[0] for r in resultados_hibridos]
df_completo['Metodo'] = [r[1] for r in resultados_hibridos]
df_completo['Confianca_Final'] = [r[2] for r in resultados_hibridos]

# 7.3. Ajustar os comentários vazios/ausentes
mask_vazios = (df_completo['Msg_comentário'].isna()) | (df_completo['Msg_comentário'].str.strip() == '')
df_completo.loc[mask_vazios, 'Sentimento_Final'] = 'NEUTRO_VAZIO'
df_completo.loc[mask_vazios, 'Metodo'] = 'SEM_COMENTARIO'
df_completo.loc[mask_vazios, 'Confianca_Final'] = 1.0


# --- PASSO 8: SALVAR E ESTATÍSTICAS FINAIS ---
caminho_final_resultados = r'C:\Users\Natasha\Documents\GITHUB\olist\resultados_finais_HIBRIDOS.xlsx'
df_completo.to_excel(caminho_final_resultados, index=False)

print(f"✅ Resultados HÍBRIDOS (ML corrigido) salvos: {caminho_final_resultados}")

# Estatísticas Finais
print("\n" + "="*70)
print("📊 ESTATÍSTICAS FINAIS DO MODELO HÍBRIDO")
print("="*70)

# Distribuição dos comentários válidos (excluindo neutro vazio)
df_validos = df_completo[df_completo['Sentimento_Final'] != 'NEUTRO_VAZIO']
distrib = df_validos['Sentimento_Final'].value_counts()
total_classificado = distrib.sum()
print(f"Distribuição dos {total_classificado:,} comentários classificados:")

# Contagem de correções por regra
correcoes = df_validos[df_validos['Metodo'] != 'ML']['Metodo'].value_counts()
total_correcoes = correcoes.sum()

for cat, qtd in distrib.items():
    perc = qtd / total_classificado * 100
    print(f"  {cat}: {qtd:,} ({perc:.1f}%)")

print(f"\n⚙️ Total de Correções por Regra (Override): {total_correcoes:,} ({total_correcoes/total_classificado*100:.1f}%)")
print(correcoes.to_string())

conf_media = df_validos['Confianca_Final'].mean()
print(f"\n🎭 Confiança Média Final: {conf_media:.1%}")

print("\n" + "="*70)
print("🎉 PROCESSO HÍBRIDO CONCLUÍDO COM SUCESSO!")
print("======================================================================\n")


🤖 TREINANDO E APLICANDO MODELO HÍBRIDO (ML + REGRAS DEFINITIVAS)
Total de amostras de treino para ML: 584
✅ Acurácia do ML puro (Teste/Validação): 80.3%
💾 Modelo ML puro salvo em: C:\Users\Natasha\Documents\GITHUB\olist\modelo_400_classificacoes_hibrido.pkl

⚙️ APLICANDO MODELO HÍBRIDO (ML + REGRAS) AO DATASET COMPLETO
✅ Resultados HÍBRIDOS (ML corrigido) salvos: C:\Users\Natasha\Documents\GITHUB\olist\resultados_finais_HIBRIDOS.xlsx

📊 ESTATÍSTICAS FINAIS DO MODELO HÍBRIDO
Distribuição dos 40,950 comentários classificados:
  NEGATIVO: 18,778 (45.9%)
  POSITIVO: 17,863 (43.6%)
  NEUTRO: 4,309 (10.5%)

⚙️ Total de Correções por Regra (Override): 14,280 (34.9%)
Metodo
Regra_Elogio     7847
Regra_Critica    2453
Regra_Erro       2136
Regra_Curto      1130
Regra_Atraso      644
Regra_Mas_Pos      43
Regra_Mas          27

🎭 Confiança Média Final: 81.3%

🎉 PROCESSO HÍBRIDO CONCLUÍDO COM SUCESSO!



In [1]:
# --- ANÁLISES DE BI COM MODELO TREINADO ---
import pandas as pd
import numpy as np
import pickle
import re
from collections import Counter
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("="*70)
print("📊 GERANDO ANÁLISES DE BI PARA DASHBOARD")
print("="*70)

# --- CONFIGURAÇÕES ---
CAMINHO_MODELO = r'C:\Users\Natasha\Documents\GITHUB\olist\modelo_400_classificacoes_hibrido.pkl'
CAMINHO_DADOS = r'C:\Users\Natasha\Documents\GITHUB\olist\olist_avaliacoes.xlsx'
CAMINHO_RESULTADOS = r'C:\Users\Natasha\Documents\GITHUB\olist\analises_bi_completas.xlsx'

# --- CLASSE DO CLASSIFICADOR (MESMA DO ANTERIOR) ---
class ClassificadorSentimentos:
    def __init__(self, caminho_modelo=CAMINHO_MODELO):
        with open(caminho_modelo, 'rb') as f:
            self.modelo = pickle.load(f)
        self.ml_model = self.modelo['model']
        self.vectorizer = self.modelo['vectorizer']
    
    def classificar_texto(self, texto):
        if not isinstance(texto, str) or texto.strip() == '':
            return 'NEUTRO', 'SEM_TEXTO', 1.0
        
        texto_lower = texto.lower()
        
        # Regras absolutas
        if 'não recomendo' in texto_lower:
            return 'NEGATIVO', 'REGRA_NAO_RECOMENDO', 0.98
        if 'não gostei' in texto_lower:
            return 'NEGATIVO', 'REGRA_NAO_GOSTEI', 0.97
        if 'excelente' in texto_lower:
            pos = texto_lower.find('excelente')
            if pos > 0 and not any(neg in texto_lower[:pos] for neg in ['não', 'nem', 'mas']):
                return 'POSITIVO', 'REGRA_EXCELENTE', 0.96
        if 'defeito' in texto_lower:
            return 'NEGATIVO', 'REGRA_DEFEITO', 0.93
        if any(pal in texto_lower for pal in ['atraso', 'demorou', 'demora']):
            if 'mas' in texto_lower and 'gostei' in texto_lower.split('mas')[-1]:
                return 'POSITIVO', 'REGRA_MAS_POSITIVO', 0.85
            return 'NEGATIVO', 'REGRA_ATRASO', 0.88
        
        palavras = texto_lower.split()
        if len(palavras) <= 5:
            if any(pal in texto_lower for pal in ['recebi', 'chegou', 'entregue', 'produto', 'ok']):
                return 'NEUTRO', 'REGRA_TEXTO_CURTO', 0.82
        
        # ML
        texto_proc = texto_lower
        texto_proc = re.sub(r'[^a-záàâãéêíóôõúüç0-9\s]', ' ', texto_proc)
        texto_proc = re.sub(r'\s+', ' ', texto_proc).strip()
        
        X = self.vectorizer.transform([texto_proc])
        predicao = self.ml_model.predict(X)[0]
        proba = self.ml_model.predict_proba(X)[0]
        confianca = max(proba)
        
        if confianca >= 0.7:
            return predicao, 'ML_ALTA_CONF', confianca
        else:
            return predicao, 'ML_BAIXA_CONF', confianca

# --- CARREGAR E CLASSIFICAR DADOS ---
print("📦 Carregando dados e classificando...")

# Carregar dados
df = pd.read_excel(CAMINHO_DADOS)
print(f"Total de registros: {len(df):,}")

# Separar válidos e vazios
mask_valido = df['Msg_comentário'].notna() & (df['Msg_comentário'].astype(str).str.strip() != '')
df_validos = df[mask_valido].copy()
df_vazios = df[~mask_valido].copy()

print(f"Comentários válidos: {len(df_validos):,}")
print(f"Vazios: {len(df_vazios):,}")

# Classificar
classificador = ClassificadorSentimentos()
resultados = []

for i, row in df_validos.iterrows():
    texto = str(row['Msg_comentário'])
    sentimento, metodo, confianca = classificador.classificar_texto(texto)
    resultados.append((sentimento, metodo, confianca))
    
    # Progresso
    if (i + 1) % 5000 == 0:
        print(f"  Classificados: {i + 1:,}/{len(df_validos):,}")

# Adicionar aos válidos
df_validos['Sentimento'] = [r[0] for r in resultados]
df_validos['Metodo_Classificacao'] = [r[1] for r in resultados]
df_validos['Confianca'] = [r[2] for r in resultados]

# Vazios = neutro
df_vazios['Sentimento'] = 'NEUTRO'
df_vazios['Metodo_Classificacao'] = 'SEM_COMENTARIO'
df_vazios['Confianca'] = 1.0

# Juntar tudo
df_completo = pd.concat([df_validos, df_vazios], ignore_index=True)

print(f"\n✅ Classificação concluída!")

# --- ABA 1: BASE COM SENTIMENTO ---
print("\n📋 Criando Aba 1: Base_com_Sentimento...")

# Manter colunas importantes para análise
colunas_manter = [
    'Msg_comentário', 'Sentimento', 'Metodo_Classificacao', 'Confianca'
]

# Adicionar outras colunas se existirem
colunas_existentes = df_completo.columns.tolist()
colunas_adicionais = [c for c in colunas_existentes 
                     if c not in colunas_manter and c != 'Unnamed: 0']

df_base = df_completo[colunas_manter + colunas_adicionais].copy()

# --- ABA 2: TOP 10 RECLAMAÇÕES ---
print("📋 Criando Aba 2: Top_10_Reclamacoes...")

# Filtrar negativos
df_negativos = df_validos[df_validos['Sentimento'] == 'NEGATIVO'].copy()

# Limpar texto para análise
def limpar_texto_analise(texto):
    texto = str(texto).lower()
    texto = re.sub(r'[^a-záàâãéêíóôõúüç0-9\s]', ' ', texto)
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

df_negativos['texto_limpo'] = df_negativos['Msg_comentário'].apply(limpar_texto_analise)

# Identificar palavras-chave de problemas
palavras_chave_problemas = {
    'atraso': ['atraso', 'demorou', 'demora', 'tarde', 'prazo'],
    'defeito': ['defeito', 'quebrado', 'quebrou', 'quebrada', 'quebrados'],
    'entregue': ['não entregue', 'não chegou', 'não recebi'],
    'qualidade': ['ruim', 'péssimo', 'horrível', 'baixa qualidade', 'qualidade ruim'],
    'errado': ['errado', 'diferente', 'outro modelo', 'não é o que comprei'],
    'faltando': ['faltando', 'só veio', 'não veio', 'incompleto'],
    'frete': ['frete caro', 'transporte', 'entrega cara'],
    'embalagem': ['embalagem', 'amassado', 'rasgado', 'avariado'],
    'contato': ['não responde', 'não atende', 'sem resposta', 'contato'],
    'cancelamento': ['cancelar', 'devolução', 'reembolso', 'troca']
}

# Contar ocorrências de cada tipo de problema
problemas_contagem = {}
for problema, palavras in palavras_chave_problemas.items():
    contagem = 0
    for texto in df_negativos['texto_limpo']:
        if any(palavra in texto for palavra in palavras):
            contagem += 1
    problemas_contagem[problema] = contagem

# Criar DataFrame de problemas
df_problemas = pd.DataFrame({
    'Tipo_Problema': list(problemas_contagem.keys()),
    'Quantidade': list(problemas_contagem.values()),
    'Percentual': [q/len(df_negativos)*100 for q in problemas_contagem.values()]
}).sort_values('Quantidade', ascending=False).head(10)

# Exemplos de cada problema
exemplos_problemas = []
for problema in df_problemas['Tipo_Problema'].head(5):
    exemplos = df_negativos[
        df_negativos['texto_limpo'].str.contains('|'.join(palavras_chave_problemas[problema]))
    ].head(3)
    
    for _, row in exemplos.iterrows():
        exemplos_problemas.append({
            'Tipo_Problema': problema,
            'Exemplo': row['Msg_comentário'][:150] + ('...' if len(row['Msg_comentário']) > 150 else ''),
            'Confianca_Classificacao': f"{row['Confianca']:.1%}"
        })

df_exemplos_problemas = pd.DataFrame(exemplos_problemas)

# --- ABA 3: RESUMO SENTIMENTO ---
print("📋 Criando Aba 3: Resumo_Sentimento...")

# Estatísticas gerais
total_geral = len(df_completo)
total_validos = len(df_validos)
total_vazios = len(df_vazios)

# Distribuição de sentimentos (apenas válidos)
distrib_sentimentos = df_validos['Sentimento'].value_counts()
distrib_percentual = df_validos['Sentimento'].value_counts(normalize=True) * 100

# Métodos de classificação
distrib_metodos = df_validos['Metodo_Classificacao'].value_counts()
distrib_metodos_perc = df_validos['Metodo_Classificacao'].value_counts(normalize=True) * 100

# Confiança média por sentimento
confianca_media = df_validos.groupby('Sentimento')['Confianca'].mean()
confianca_geral = df_validos['Confianca'].mean()

# Criar DataFrame resumo
df_resumo = pd.DataFrame({
    'Metrica': [
        'Total Registros', 'Comentários Válidos', 'Registros Sem Comentário',
        'POSITIVO (Quantidade)', 'POSITIVO (%)', 
        'NEGATIVO (Quantidade)', 'NEGATIVO (%)',
        'NEUTRO (Quantidade)', 'NEUTRO (%)',
        'Confiança Média Geral', 'Confiança Média POSITIVO',
        'Confiança Média NEGATIVO', 'Confiança Média NEUTRO',
        'Acurácia Modelo (Treino)', 'Data Processamento'
    ],
    'Valor': [
        f"{total_geral:,}",
        f"{total_validos:,}",
        f"{total_vazios:,}",
        f"{distrib_sentimentos.get('POSITIVO', 0):,}",
        f"{distrib_percentual.get('POSITIVO', 0):.1f}%",
        f"{distrib_sentimentos.get('NEGATIVO', 0):,}",
        f"{distrib_percentual.get('NEGATIVO', 0):.1f}%",
        f"{distrib_sentimentos.get('NEUTRO', 0):,}",
        f"{distrib_percentual.get('NEUTRO', 0):.1f}%",
        f"{confianca_geral:.1%}",
        f"{confianca_media.get('POSITIVO', 0):.1%}",
        f"{confianca_media.get('NEGATIVO', 0):.1%}",
        f"{confianca_media.get('NEUTRO', 0):.1%}",
        f"{classificador.modelo.get('acuracia', 'N/A'):.1%}",
        datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    ]
})

# --- ABA 4: TOP 20 GERAL ---
print("📋 Criando Aba 4: Top_20_Geral...")

# Função para extrair palavras-chave
def extrair_palavras_chave(texto, n_palavras=3):
    texto = str(texto).lower()
    palavras = texto.split()
    
    # Remover stopwords simples
    stopwords = ['o', 'a', 'os', 'as', 'um', 'uma', 'uns', 'umas',
                'de', 'do', 'da', 'dos', 'das', 'em', 'no', 'na',
                'nos', 'nas', 'por', 'para', 'com', 'sem', 'sob',
                'sobre', 'entre', 'que', 'e', 'mas', 'porém']
    
    palavras_filtradas = [p for p in palavras if p not in stopwords and len(p) > 2]
    
    # Pegar as n palavras mais frequentes
    if len(palavras_filtradas) >= n_palavras:
        return ' '.join(palavras_filtradas[:n_palavras])
    return ' '.join(palavras_filtradas)

# Para cada sentimento, pegar os mais representativos
top_comentarios = []

for sentimento in ['POSITIVO', 'NEGATIVO', 'NEUTRO']:
    # Pegar com maior confiança
    df_sentimento = df_validos[df_validos['Sentimento'] == sentimento]
    
    # Ordenar por confiança
    top_confianca = df_sentimento.nlargest(7, 'Confianca')
    
    for _, row in top_confianca.iterrows():
        palavras_chave = extrair_palavras_chave(row['Msg_comentário'], 3)
        top_comentarios.append({
            'Sentimento': sentimento,
            'Comentario': row['Msg_comentário'][:120] + ('...' if len(row['Msg_comentário']) > 120 else ''),
            'Palavras_Chave': palavras_chave,
            'Confianca': f"{row['Confianca']:.1%}",
            'Metodo': row['Metodo_Classificacao']
        })
    
    # Pegar mais curtos (para ver padrões)
    df_sentimento['tamanho'] = df_sentimento['Msg_comentário'].str.len()
    top_curtos = df_sentimento.nsmallest(3, 'tamanho')
    
    for _, row in top_curtos.iterrows():
        if len(row['Msg_comentário']) > 10:  # Evitar muito curtos
            palavras_chave = extrair_palavras_chave(row['Msg_comentário'], 3)
            top_comentarios.append({
                'Sentimento': sentimento,
                'Comentario': row['Msg_comentário'],
                'Palavras_Chave': palavras_chave,
                'Confianca': f"{row['Confianca']:.1%}",
                'Metodo': row['Metodo_Classificacao']
            })

df_top_geral = pd.DataFrame(top_comentarios).head(20)

# --- ABA 5: ANÁLISE CONFIANÇA ---
print("📋 Criando Aba 5: Analise_Confianca...")

# Análise de confiança por método
confianca_por_metodo = df_validos.groupby('Metodo_Classificacao').agg({
    'Confianca': ['mean', 'count'],
    'Sentimento': lambda x: x.mode()[0] if len(x.mode()) > 0 else 'N/A'
}).round(3)

confianca_por_metodo.columns = ['Confianca_Media', 'Quantidade', 'Sentimento_Mais_Comum']
confianca_por_metodo = confianca_por_metodo.reset_index()

# Distribuição de confiança
bins = [0, 0.3, 0.5, 0.7, 0.85, 1.0]
labels = ['Muito Baixa (0-30%)', 'Baixa (30-50%)', 'Media (50-70%)', 
          'Alta (70-85%)', 'Muito Alta (85-100%)']

df_validos['Faixa_Confianca'] = pd.cut(df_validos['Confianca'], bins=bins, labels=labels)
distrib_confianca = df_validos['Faixa_Confianca'].value_counts().reset_index()
distrib_confianca.columns = ['Faixa_Confianca', 'Quantidade']

# Sentimento por faixa de confiança
sentimento_por_confianca = pd.crosstab(
    df_validos['Faixa_Confianca'], 
    df_validos['Sentimento'],
    normalize='index'
).round(3) * 100

sentimento_por_confianca = sentimento_por_confianca.reset_index()

# --- SALVAR TODAS AS ABAS ---
print(f"\n💾 Salvando análises em: {CAMINHO_RESULTADOS}")

with pd.ExcelWriter(CAMINHO_RESULTADOS, engine='xlsxwriter') as writer:
    # Aba 1: Base completa com sentimentos
    df_base.to_excel(writer, sheet_name='Base_com_Sentimento', index=False)
    
    # Aba 2: Top 10 problemas
    df_problemas.to_excel(writer, sheet_name='Top_10_Reclamacoes', index=False)
    
    # Se houver exemplos, salvar em sub-aba
    if len(df_exemplos_problemas) > 0:
        df_exemplos_problemas.to_excel(writer, sheet_name='Exemplos_Problemas', index=False)
    
    # Aba 3: Resumo estatístico
    df_resumo.to_excel(writer, sheet_name='Resumo_Sentimento', index=False)
    
    # Aba 4: Top 20 geral
    df_top_geral.to_excel(writer, sheet_name='Top_20_Geral', index=False)
    
    # Aba 5: Análise de confiança
    confianca_por_metodo.to_excel(writer, sheet_name='Confianca_por_Metodo', index=False)
    distrib_confianca.to_excel(writer, sheet_name='Distribuicao_Confianca', index=False)
    sentimento_por_confianca.to_excel(writer, sheet_name='Sentimento_por_Confianca', index=False)
    
    # Formatação
    workbook = writer.book
    
    # Formato para percentuais
    percent_format = workbook.add_format({'num_format': '0.0%'})
    
    # Formatar aba de resumo
    worksheet_resumo = writer.sheets['Resumo_Sentimento']
    for row_num in range(1, len(df_resumo) + 1):
        if any(x in df_resumo.iloc[row_num-1, 0] for x in ['%', 'Confiança']):
            worksheet_resumo.write(row_num, 1, df_resumo.iloc[row_num-1, 1])
    
    # Autoajustar colunas
    for sheet_name in writer.sheets:
        worksheet = writer.sheets[sheet_name]
        for i, col in enumerate(writer.sheets[sheet_name]._shared_strings):
            column_len = max(len(str(col)), 10)
            worksheet.set_column(i, i, min(column_len, 50))

print("✅ Análises salvas com sucesso!")

# --- RELATÓRIO FINAL ---
print("\n" + "="*70)
print("📊 RELATÓRIO DAS ANÁLISES GERADAS")
print("="*70)

print(f"\n📁 ARQUIVO GERADO: {CAMINHO_RESULTADOS}")
print("\n📋 ABAS DISPONÍVEIS PARA BI:")

abas_info = [
    ("1. Base_com_Sentimento", "Base completa com classificações", "Filtros, gráficos gerais"),
    ("2. Top_10_Reclamacoes", "Principais tipos de problemas", "Dashboard de melhorias"),
    ("3. Resumo_Sentimento", "Métricas e estatísticas", "KPIs e resumo executivo"),
    ("4. Top_20_Geral", "Exemplos representativos", "Validação e exemplos"),
    ("5. Confianca_por_Metodo", "Qualidade das classificações", "Monitoramento do modelo"),
    ("6. Exemplos_Problemas", "Exemplos de cada problema", "Análise qualitativa")
]

for nome, descricao, uso in abas_info:
    print(f"\n   📄 {nome}")
    print(f"      Descrição: {descricao}")
    print(f"      Uso no BI: {uso}")

print(f"\n📈 ESTATÍSTICAS PRINCIPAIS:")
print(f"   • Total classificados: {total_validos:,}")
print(f"   • Distribuição: {distrib_sentimentos.get('POSITIVO', 0):,} POS, "
      f"{distrib_sentimentos.get('NEGATIVO', 0):,} NEG, "
      f"{distrib_sentimentos.get('NEUTRO', 0):,} NEU")
print(f"   • Confiança média: {confianca_geral:.1%}")
print(f"   • Principais problemas: {', '.join(df_problemas['Tipo_Problema'].head(3).tolist())}")

print("""
======================================================================
🎯 COMO USAR NO BI:
======================================================================

1. 📊 POWER BI / TABLEAU:
   • Conecte ao arquivo Excel
   • Use 'Base_com_Sentimento' como tabela fato
   • Use outras abas como dimensões

2. 📈 DASHBOARDS SUGERIDOS:
   • Painel de Satisfação (NPS virtual)
   • Mapa de Problemas Recorrentes
   • Evolução Temporal (se tiver datas)
   • Top Lojas/Vendedores (se tiver esses dados)

3. 🔔 ALERTAS AUTOMÁTICOS:
   • Aumento súbito de negativos
   • Novos tipos de problemas
   • Queda na confiança do modelo

4. 📋 AÇÕES RECOMENDADAS:
   • Analisar Top_10_Reclamacoes mensalmente
   • Validar Top_20_Geral para checar qualidade
   • Monitorar Confianca_por_Metodo

💡 DICA: Atualize mensalmente com novos dados!
   Basta rodar este código novamente.
""")

# --- CÓDIGO PARA ATUALIZAÇÃO MENSAL ---
print("\n" + "="*70)
print("🔄 CÓDIGO PARA ATUALIZAÇÃO MENSAL")
print("="*70)

codigo_atualizacao = '''
# Para atualizar mensalmente, basta:
# 1. Colocar novos dados em olist_avaliacoes.xlsx
# 2. Executar este script novamente

# Ou criar um script automático:
import schedule
import time

def atualizar_analises_mensal():
    print(f"Atualizando análises em {datetime.now()}")
    # Coloque todo o código acima aqui
    print("✅ Análises atualizadas!")

# Agendar para todo dia 1º do mês
schedule.every().month.at("08:00").do(atualizar_analises_mensal)

print("Agendado atualização mensal...")
while True:
    schedule.run_pending()
    time.sleep(3600)  # Verifica a cada hora
'''

print(codigo_atualizacao)
print("\n✅ PRONTO PARA USAR NO SEU BI! 🚀")

📊 GERANDO ANÁLISES DE BI PARA DASHBOARD
📦 Carregando dados e classificando...
Total de registros: 99,224
Comentários válidos: 40,950
Vazios: 58,274
  Classificados: 60,000/40,950
  Classificados: 65,000/40,950
  Classificados: 70,000/40,950
  Classificados: 75,000/40,950
  Classificados: 80,000/40,950
  Classificados: 85,000/40,950
  Classificados: 90,000/40,950
  Classificados: 95,000/40,950

✅ Classificação concluída!

📋 Criando Aba 1: Base_com_Sentimento...
📋 Criando Aba 2: Top_10_Reclamacoes...
📋 Criando Aba 3: Resumo_Sentimento...
📋 Criando Aba 4: Top_20_Geral...
📋 Criando Aba 5: Analise_Confianca...

💾 Salvando análises em: C:\Users\Natasha\Documents\GITHUB\olist\analises_bi_completas.xlsx


AttributeError: 'Worksheet' object has no attribute '_shared_strings'

In [2]:
# --- DASHBOARD DE AÇÕES BASEADO NAS RECLAMAÇÕES ---
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

print("="*70)
print("🎯 DASHBOARD DE AÇÕES - PRIORIDADES IDENTIFICADAS")
print("="*70)

# Seus dados
dados_problemas = {
    'Tipo_Problema': ['atraso', 'entregue', 'cancelamento', 'errado', 'defeito', 
                     'faltando', 'contato', 'qualidade', 'embalagem'],
    'Quantidade': [2695, 1902, 1054, 798, 649, 616, 612, 445, 371],
    'Percentual': [15.32, 10.81, 5.99, 4.54, 3.69, 3.50, 3.48, 2.53, 2.11]
}

df_problemas = pd.DataFrame(dados_problemas)

# --- ABA 1: RESUMO EXECUTIVO ---
print("\n📋 RESUMO EXECUTIVO PARA GESTORES:")
print("-" * 50)

for _, row in df_problemas.iterrows():
    if row['Percentual'] > 5:
        print(f"🚨 {row['Tipo_Problema'].upper()}: {row['Percentual']:.1f}%")
        print(f"   → {row['Quantidade']} ocorrências")
        print(f"   → Ação prioritária necessária")
    elif row['Percentual'] > 3:
        print(f"⚠️  {row['Tipo_Problema'].upper()}: {row['Percentual']:.1f}%")
        print(f"   → {row['Quantidade']} ocorrências")
        print(f"   → Monitorar mensalmente")

# --- ABA 2: PLANO DE AÇÃO ---
print("\n📋 PLANO DE AÇÃO RECOMENDADO:")
print("-" * 50)

acoes_recomendadas = {
    'atraso': [
        "Implementar sistema de tracking em tempo real",
        "Rever prazos de entrega divulgados",
        "Parceria com transportadoras mais eficientes",
        "SLA de entrega por região"
    ],
    'entregue': [
        "Sistema de confirmação de entrega com foto",
        "Notificação ao cliente quando marcado como entregue",
        "Protocolo para entregas não recebidas",
        "Treinamento de entregadores"
    ],
    'cancelamento': [
        "Análise de motivo de cancelamentos",
        "Pesquisa pós-cancelamento",
        "Política de retenção para cancelamentos",
        "Checkout otimizado"
    ],
    'errado': [
        "Controle de qualidade na separação",
        "Checklist de conferência",
        "Sistema de código de barras",
        "Treinamento de equipe de warehouse"
    ],
    'defeito': [
        "Garantia estendida",
        "Processo rápido de troca",
        "Parceria com assistência técnica",
        "Controle de qualidade de fornecedores"
    ]
}

for problema, acoes in acoes_recomendadas.items():
    if problema in df_problemas['Tipo_Problema'].values:
        percentual = df_problemas[df_problemas['Tipo_Problema'] == problema]['Percentual'].values[0]
        print(f"\n🎯 {problema.upper()} ({percentual:.1f}%):")
        for i, acao in enumerate(acoes, 1):
            print(f"   {i}. {acao}")

# --- ABA 3: KPIs DE MELHORIA ---
print("\n📋 KPIs PARA MONITORAR MELHORIA:")
print("-" * 50)

kpis = [
    ("Redução de reclamações por atraso", "Métrica: % redução mês a mês", "Meta: -20% em 3 meses"),
    ("Taxa de entregas confirmadas", "Métrica: % entregas com confirmação", "Meta: 95%"),
    ("Taxa de cancelamento", "Métrica: % pedidos cancelados", "Meta: <3%"),
    ("Satisfação pós-troca", "Métrica: NPS após troca", "Meta: >70"),
    ("Tempo de resposta a contatos", "Métrica: Horas para resposta", "Meta: <24h")
]

for kpi, metrica, meta in kpis:
    print(f"📊 {kpi}")
    print(f"   {metrica}")
    print(f"   🎯 {meta}")

# --- SALVAR DASHBOARD COMPLETO ---
print("\n💾 Criando dashboard completo em Excel...")

with pd.ExcelWriter(r'C:\Users\Natasha\Documents\GITHUB\olist\dashboard_acoes.xlsx', 
                    engine='xlsxwriter') as writer:
    
    # Aba 1: Problemas e percentuais
    df_problemas.to_excel(writer, sheet_name='Top_Problemas', index=False)
    
    # Aba 2: Plano de ação
    dados_acoes = []
    for problema, acoes in acoes_recomendadas.items():
        for acao in acoes:
            dados_acoes.append({'Problema': problema.upper(), 'Acao': acao, 'Status': 'Pendente'})
    df_acoes = pd.DataFrame(dados_acoes)
    df_acoes.to_excel(writer, sheet_name='Plano_Acao', index=False)
    
    # Aba 3: KPIs
    df_kpis = pd.DataFrame(kpis, columns=['KPI', 'Metrica', 'Meta'])
    df_kpis.to_excel(writer, sheet_name='KPIs_Monitoramento', index=False)
    
    # Aba 4: Evolução recomendada
    evolucao = pd.DataFrame({
        'Mes': ['Mês 1', 'Mês 2', 'Mês 3', 'Mês 4', 'Mês 5', 'Mês 6'],
        'Meta_Atraso': [15.3, 12.0, 9.5, 7.5, 6.0, 5.0],
        'Meta_Entregue': [10.8, 8.5, 6.5, 5.0, 4.0, 3.0],
        'Meta_Cancelamento': [6.0, 5.0, 4.0, 3.5, 3.0, 2.5]
    })
    evolucao.to_excel(writer, sheet_name='Evolucao_Metas', index=False)
    
    # Aba 5: Responsáveis
    responsaveis = pd.DataFrame({
        'Problema': ['ATRASO', 'ENTREGUE', 'CANCELAMENTO', 'ERRADO', 'DEFEITO'],
        'Responsavel_Principal': ['Logística', 'TI', 'Vendas', 'Qualidade', 'Pós-Venda'],
        'Responsavel_Secundario': ['Compras', 'Atendimento', 'Marketing', 'Warehouse', 'Fornecedores'],
        'Prazo': ['3 meses', '2 meses', '2 meses', '4 meses', '3 meses']
    })
    responsaveis.to_excel(writer, sheet_name='Responsaveis', index=False)

print("✅ Dashboard salvo: dashboard_acoes.xlsx")

print("""
======================================================================
🎯 RESUMO DAS DESCOBERTAS:
======================================================================

🏆 TOP 3 PROBLEMAS (67% do esforço deve ser aqui):
1. ATRASO (15.3%) → Problema de LOGÍSTICA
2. ENTREGUE (10.8%) → Problema de SISTEMA
3. CANCELAMENTO (6.0%) → Problema de VENDAS

📊 REGRA 80/20 APLICADA:
• Top 3 problemas = 32.1% de TODAS as reclamações
• Focar nesses resolve 1/3 dos problemas dos clientes

💡 INSIGHTS VALIOSOS:
• Clientes estão MUITO insatisfeitos com prazos
• Sistema de entrega tem falhas graves
• Processo de cancelamento precisa de atenção

🚀 PRÓXIMOS PASSOS:
1. Apresente esses dados para a diretoria
2. Crie squad para cada problema prioritário
3. Monitore redução mensalmente

🎯 SEU BI AGORA TEM:
• Dados para tomar decisões baseadas em evidências
• Plano de ação claro
• KPIs para medir sucesso
""")

🎯 DASHBOARD DE AÇÕES - PRIORIDADES IDENTIFICADAS

📋 RESUMO EXECUTIVO PARA GESTORES:
--------------------------------------------------
🚨 ATRASO: 15.3%
   → 2695 ocorrências
   → Ação prioritária necessária
🚨 ENTREGUE: 10.8%
   → 1902 ocorrências
   → Ação prioritária necessária
🚨 CANCELAMENTO: 6.0%
   → 1054 ocorrências
   → Ação prioritária necessária
⚠️  ERRADO: 4.5%
   → 798 ocorrências
   → Monitorar mensalmente
⚠️  DEFEITO: 3.7%
   → 649 ocorrências
   → Monitorar mensalmente
⚠️  FALTANDO: 3.5%
   → 616 ocorrências
   → Monitorar mensalmente
⚠️  CONTATO: 3.5%
   → 612 ocorrências
   → Monitorar mensalmente

📋 PLANO DE AÇÃO RECOMENDADO:
--------------------------------------------------

🎯 ATRASO (15.3%):
   1. Implementar sistema de tracking em tempo real
   2. Rever prazos de entrega divulgados
   3. Parceria com transportadoras mais eficientes
   4. SLA de entrega por região

🎯 ENTREGUE (10.8%):
   1. Sistema de confirmação de entrega com foto
   2. Notificação ao cliente quando